# Анализ данных на Python

*Алла Тамбовцева, НИУ ВШЭ*

## Примеры работы с API ВКонтакте

## Подготовка к работе

### Знакомимся с документацией API

Для начала давайте посмотрим на [документацию](https://dev.vk.com/ru/api/api-requests) API ВКонтакте и посмотрим, как формировать запросы для получения информации. 

Общий вид запроса следующий:

    https://api.vk.com/method/<METHOD>?<PARAMS>
    
Вместо `<METHOD>` указывается название метода, вместо `<PARAMS>` – параметры этого метода в виде пар `<параметр>=<значение>`, объединённых через `&`. Какие-то параметры являются необязательными, какие-то – обязательными. Обязательными для сбора данных являются два параметра:

* токен доступа `access_token`;
* версия API `v`.

Примеры методов и параметров (вместо токена доступа число 1):

<table>
    <tr>
        <th>Запрос</th><th>Ссылка</th>
    </tr>
    <tr>
        <td>базовая информация по пользователю с id=743784474</td>
        <td>https://api.vk.com/method/users.get?user_ids=743784474&access_token=1&v=5.199</td>
    </tr>
    <tr>
        <td>дата рождения пользователя с id=743784474</th>
        <td>https://api.vk.com/method/users.get?user_ids=743784474&fields=bdate&access_token=1&v=5.199</td>
    </tr>
    <tr>
        <td>базовая информация по пользователям с id=743784474 и id=20473269</td>
        <td>https://api.vk.com/method/users.get?user_ids=743784474,20473269&fields=bdate&access_token=1&v=5.199</td>
    </tr>
    <tr>
        <td>100 последних постов со стены сообщества с названием hse</td>
        <td>https://api.vk.com/method/wall.get?domain=hse&count=100&access_token=1&v=5.199</td>
        </tr>
</table>

Итак, глобальная задача – получить токен доступа, сформировать ссылку для нужного запроса, отправить запрос к серверу и обработать полученный результат. Посмотрим, как отправляются запросы к серверу в Python.

### Модуль `requests` для отправки запросов

Базовый модуль `requests` в Python служит для отправки запросов типа `GET` или `POST`. В нашем случае достаточно запроса `GET` для получения информации, так как отправлять информацию на сервер нам не нужно (мы не пишем бота и не планируем удалённо управлять своим аккаунтом). Импортируем модуль, вызовем оттуда функцию `get()` и отправим какой-нибудь запрос из примеров выше:

In [ ]:
import requests

In [ ]:
req = requests.get("https://api.vk.com/method/users.get?user_ids=743784474&access_token=1&v=5.199")
print(req)

В переменной `req` хранится объект типа `Response`, он скрыт, временно хранится в некоторой ячейке памяти. Код 200 означает, что запрос отправлен и ответ от сервера получен, это стандартный код [состояния HTTP](https://ru.wikipedia.org/wiki/%D0%A1%D0%BF%D0%B8%D1%81%D0%BE%D0%BA_%D0%BA%D0%BE%D0%B4%D0%BE%D0%B2_%D1%81%D0%BE%D1%81%D1%82%D0%BE%D1%8F%D0%BD%D0%B8%D1%8F_HTTP). Этот код можно извлечь отдельно, вызвав атрибут `status_code`, это может пригодится для последующей работы (если код равен 200, продолжаем работу, если нет, реализуем программу по поиску ошибки и её причины):

In [ ]:
# код типа integer
print(req.status_code, type(req.status_code))

Из `req` можем извлечь ссылку, ту, что в адресной строке (сейчас не так интересно, потому что мы ссылку сами отправили, но может пригодится):

In [ ]:
print(req.url)

А можем сам результат в виде строки, в данном случае это текст, который мы видим на странице при переходе по ссылке выше в браузере:

In [ ]:
# строка, тип str
print(req.text, type(req.text))

Готового результата с информацией по пользователю с указанным id мы не получили, поскольку токен доступа был указан некорректный. Однако видно, что, в любом случае, результат извлекается, он непустой и представлен в виде JSON-строки. Такую строку можно обработать с помощью модуля `json`, в нём есть функция `loads()` для десериализации JSON-строки и превращения её в питоновский словарь. Но мы поступим проще – применим метод `.json()`, который определён на объектах класса `Response`:

In [ ]:
# словарь, тип dict
print(req.json(), type(req.json()))

В результате мы получили готовый словарь, из которого можно извлекать необходимую информацию по ключам! Однако для того, чтобы информация в словаре была, нужно получить токен доступа для работы.

### Получаем токен доступа

Итак, алгоритм получения токена доступа:

1. Создаём новое приложение в разделе *Мои приложения* (https://vk.com/apps?act=manage) типа *Standalone-приложение* (поле *Платформа*).

2. Переходим в предлагаемый сервис для авторизации, при первой работе с сервисом вписываем свои данные (относительно новая политика безопасности vk), выбираем *Add app*, вписываем название приложения, в качестве платформы выбираем *Web*, в качестве домена и ссылки можно вписать предлагаемые для примера `mysite.com` и `https://mysite.com`, так как само приложение, связанное с каким-то своим сервером нам не нужно, нужен только ключ для последующего сбора данных.

3. Когда приложение создано, копируем его числовой ID (*App ID* из *App information*) и сохраняем. 

4. Для получения токена доступа формируем ссылку вида:

        https://oauth.vk.com/authorize?client_id=i&display=page&redirect_uri=http://oauth.vk.com/blank.html&scope=all&response_type=token

и вместо `i` в `client_id` подставляем туда ID приложения. Переходим по ссылке, из адресной строки копируем токен доступа – набор символов после `access_token` и до `&expires_in`. Никому не показываем токен, так как он даёт доступ к вашему аккаунту. Токен действителен в течение суток при работе с того же IP адреса.

Для более удобного автоматизированного получения токена написан код ниже:

In [ ]:
from getpass import getpass

In [ ]:
# getpass() – аналог input(), скрывает вводимый текст
# подставляем id приложения и переходим по ссылке

app_id = getpass("Enter your client id: ")
url = f"https://oauth.vk.com/authorize?client_id={app_id}&display=page&redirect_uri=http://oauth.vk.com/blank.html&scope=all&response_type=token"
print(url)

In [ ]:
# копируем ссылку из адресной строки (длинная с access_token)
# разбиваем ссылку на части и извлекаем токен -> token

full_link = getpass()
token = full_link.split("access_token=")[1].split("&")[0]

Теперь приступаем к работе!

## Выгружаем посты со стены сообщества

На этом практическом занятии мы будем выгружать посты из сообщества скалодрома [Rock Zona](https://vk.com/rzclimbing). Сохраним в переменные версию API, ссылку для метода работы со стеной сообщества и короткое название сообщества:

In [ ]:
v = "5.199"
main_wall = "https://api.vk.com/method/wall.get"
domain = "rzclimbing"

Чтобы подставить необходимые параметры метода `wall.get`, а также токен доступа, можно воспользоваться обычным форматированием строк или f-строками, однако есть ещё более удобный способ. Функция `get()` из `requests` умеет подставлять в запрос необходимые параметры и объединять их с помощью `?` и `&`. Сохраним необходимые параметры в виде словаря:

In [ ]:
# 100 – максимальное число постов за раз

params_wall = {"access_token" : token, 
              "domain" : domain, 
              "count" : 100,
              "v" : v}

Теперь сформируем запрос:

In [ ]:
req_wall = requests.get(main_wall, params = params_wall)
# print(req_wall.url)

Извлечём результаты и преобразуем JSON-строку в словарь:

In [ ]:
json_wall = req_wall.json()

### Задача 1

Извлеките из `json_wall` общее число постов на стене и сохраните его в переменную `nposts`. Извлеките из `json_wall` список словарей с информацией о каждом извлечённом после и сохраните его в переменную `items_wall`.

In [ ]:
### YOUR CODE HERE ###

### Задача 2

Выберите первый элемент списка `items_wall`, назовите его `i`. Извлеките из элемента `i` следующие компоненты:

* id поста;
* дата поста;
* текст поста;
* число лайков;
* число просмотров;
* число комментариев.

In [ ]:
### YOUR CODE HERE ###

### Задача 3

Изучить один пост и понять, что нам от него нужно, это хорошо, но, конечно, мы захотим выгрузить все посты сразу, а уже потом разобраться, какую информацию о них нам оставить. Ограничения данного API таковы, что за один раз мы можем выгрузить только 100 постов. Хорошие новости: каждый раз при выгрузке мы можем начинать с того поста, на котором закончили, то есть сначала выгрузить первые 100 постов, потом – следующие 100 постов, и так до тех пор, пока не заполучим все.

Общее число постов сохранено в `nposts`. Посчитайте, сколько раз нужно будет выполнить выгрузку по 100 постов, чтобы собрать все тексты, и сохраните его в переменную `iterate`.

In [ ]:
### YOUR CODE HERE ###

### Задача 4

Прочитайте в документации к API ВКонтакте про аргумент `offset` в методе `wall.get`. Используя полученную информацию и блоки кода ниже, выгрузите и сохраните в список `items_all` данные по всем постам на стене сообщества.

**Подсказка:** чтобы расширять список правильным образом, используйте метод `.extend()`, а не `.append()`, он добавляет не один элемент, а сразу несколько.

In [ ]:
params_wall_long = {"access_token" : token, 
                    "domain" : domain, 
                    "count" : 100,
                    "v" : v,
                    "offset" : 0}

In [ ]:
import time

In [ ]:
# функция sleep() из time выставляет задержку запуска кода
# здесь задержка в 1.2 секунды

items_all = []

for i in range(iterate):
    req_wall_long = requests.get(main_wall, params = params_wall_long)
    json_wall_long = req_wall_long.json()
    items_wall_long = json_wall_long['response']['items']
    
    ### YOUR CODE HERE ###
    
    time.sleep(1.2)
    print(i)

Проверьте длину списка `items_all` – все ли посты собраны. Преобразуйте полученный результат – список словарей – в датафрейм Pandas.

In [ ]:
### YOUR CODE HERE ###

### Задача 5

Создайте на основе полученного датафрейма новый датафрейм `small` со следующими столбцами:

* id поста (`id`);
* дата поста (`date`);
* текст поста (`text`);
* число лайков (`nlikes`);
* число просмотров (`nviews`);
* число комментариев (`ncomments`).

**Подсказка:** отберите сначала все столбцы с нужной информацией, проверьте, есть ли в датафрейме пропущенные значения, напишите функцию для извлечения только числа лайков/просмотров/комментариев и примените её, удалите лишние столбцы. 

In [ ]:
### YOUR CODE HERE ###

### Задача 6

Добавьте в `small` столбец `date_time` с датой-временем поста в формате `datetime`.

In [ ]:
### YOUR CODE HERE ###

### Задача 7

Выберите только те строки в полученном датафрейме, которые соответствуют постам с числом комментариев больше 0, и сохраните их в датафрейм `with_comm`.

In [ ]:
### YOUR CODE HERE ###

Если успеем вместе – код для бонусной части для сбора комментариев:

In [ ]:
with_comm10 = with_comm.head(10)
ids = list(with_comm10["id"])

In [ ]:
comments_all = []

for i in ids:
    params_comm = {"owner_id" : -38936316,
              "post_id" : i,
              "v" : v,
              "access_token" : token,
              "count" : 100, 
              "thread_items_count" : 10}
    req = requests.get(main_comm, params = params_comm)
    comm_json = req.json() 
    comm_list = comm_json["response"]["items"]
    comments_all.extend(comm_list)
    time.sleep(1)

In [ ]:
df_comm = pd.DataFrame(comments_all)
df_comm.head(3)

In [ ]:
with_comm10.rename(columns = {"id" : "post_id"}, inplace = True)
with_comm10.head(3)

In [ ]:
final = with_comm10.merge(df_comm, on = "post_id", how = "left")